### アノテーション保存用のカラム追加

#### docsテーブル確認
 * ディレクトリ：../../NaturalLanguageProcessing/data/sqlite3/sqlite3
 * テーブル名：docs

| #  | 項目                                        | 型                                   | 項目説明                                       |
| -- | ------------------------------------------- | ------------------------------------ | -----------------------------------------------|
| 1  | id                                          | INTEGER                              | 既に作成済                                     |
| 2  | content                                     | TEXT                                 | 既に作成済                                     |
| 3  | meta_info                                   | BLOB                                 | 既に作成済                                     |
| 4  | sentence                                    | BLOB                                 | 既に作成済                                     |
| 5  | chunk                                       | BLOB                                 | 既に作成済                                     |
| 6  | token                                       | BLOB                                 | 既に作成済                                     |
| 7  | <font color="red"><b>affiliation</b></font> | <font color="red"><b>BLOB</b></font> | <font color="red"><b>今回カラム追加</b></font> |


### データベース接続の関数定義

In [2]:
import json
import sqlite3

conn = None

# データベース接続
def connect():
    # global変数でconnを呼び出し
    global conn
    # データベースの場所を指定
    conn = sqlite3.connect('../../NaturalLanguageProcessing/data/sqlite3/sqlite3')

# データベース接続終了
def close():
#   終了
    conn.close()

# テーブル作成
def create_table():
#   executeでSQL構文作成、docsがあれば削除
    conn.execute('DROP TABLE IF EXISTS docs')
#   docsテーブルを新規作成
    conn.execute('''CREATE TABLE docs (
            id          INTEGER PRIMARY KEY AUTOINCREMENT,
            content     TEXT,
            meta_info   BLOB,
            sentence    BLOB,
            chunk       BLOB,
            token       BLOB
        )''')

# データをインサートする
def db_load(values):
    
#   valuesに指定したパラメータ順序またはマッピングを?に入れて実行する
    conn.executemany(
        'INSERT INTO docs (content, meta_info) VALUES (?,?)',
        values)
#   確定
    conn.commit()

# 一部のデータを見る
def get(doc_id, fl):
#   .fetchone()でカーソルの次の行を取得
    row_ls = conn.execute(f"SELECT {','.join(fl)} FROM docs WHERE id = {doc_id}").fetchone()
    row_dict = {}
#   flとrow_lsで抜き出したデータをzipする
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

# id番号を抜き出す
def get_all_ids(limit, offset=0):
#   limitで取得上限、OFFSETで開始位置を指定してデータを抜き出す。そのデータの1番目id番号を抜き出す
    return [record[0] for record in conn.execute('SELECT id FROM docs LIMIT ? OFFSET ?', (limit, offset))]

#### カラム追加（affiliation）


In [ ]:
# 作成していないときに実行
# connect()

# name = "affiliation"
# conn.execute("")
# conn.execute("ALTER TABLE docs ADD COLUMN '{0}' 'BLOB'".format(name))

# close()

### 正規表現のパターンによるテキストデータの解析

| 記法      | 意味                                                                |
|-----------| ------------------------------------------------------------------- |
| `.`       | 任意の1文字                                                         |
| `*`       | 直前の文字の0回以上の繰り返し                                       |
| `+`       | 直前の文字の1回以上の繰り返し                                       |
| `*?`      | 直前の文字の0回以上の繰り返し、ただし最小部分にマッチ（最小マッチ） |
| `+?`      | 直前の文字の1回以上の繰り返し、ただし最小部分にマッチ（最小マッチ） |
| `[^ABC]`  | A、B、C以外の1文字                                                  |


### アノテーション用関数定義

#### アノテーションとは

正規表現のパターンを用いてテキストデータ中の名詞句に意味付けを行うこと。

In [3]:
# 大学・学会のアノテーションを付けるプログラム
def create_annotation(doc_id, ptn):
    # SQLiteからdoc_idに該当する行のテキストを取得
    row = get(doc_id, fl=['content'])
    text = row['content']
    annos = []
    
    for chunk in get_annotation(doc_id, 'chunk'):
        # チャンクのbeginからendまでのtextを取り出し、正規表現でマッチングをかける
        chunk_str = text[chunk['begin']:chunk['end']]
        m = ptn.search(chunk_str)
        if not m:
            continue  
        # マッチしたら開始位置と終了位置を取得
        anno = {
            'begin':    chunk['begin'] + m.start(),
            'end':      chunk['begin'] + m.end(),
        }
        print(text[anno['begin']:anno['end']])
        annos.append(anno)
    return annos

# アノテーションを設定
def set_annotation(doc_id, name, value):
#   docsのid行をwhere idで指定し、その行にname = valueのアノテーションを追加　
    conn.execute('UPDATE docs SET {0} = ? where id = ?'.format(name), (json.dumps(value), doc_id))
#   確定
    conn.commit()

# アノテーションを確認
def get_annotation(doc_id, name):
#   docsのid行をwhere idで指定しnameから取り出す
    row = conn.execute('SELECT {0} FROM docs WHERE id = ?'.format(name), (doc_id,)).fetchone()
    if row[0] is not None:
        return json.loads(row[0])
    else:
        return []

### docsテーブルの「affiliation」カラムにデータ追加


In [4]:
import re

# dict型で大学と学会の正規表現を定義する
dic = [
    r'[^ 『（ ]+?大学',
    r'[^ 『（ ]+?学会',
    r'[^ 『（ ]+?協会',
]

# 上記dictを|でつないで正規表現としてコンパイルする。|はorとして機能する。
ptn = re.compile(r'|'.join(dic))

anno_name = 'affiliation'

connect()
for doc_id in get_all_ids(limit=-1):
    # create_annotationでアノテーションを生成
    annos = create_annotation(doc_id, ptn)
    print(annos)
    # set_annotationでSQLiteに生成したアノテーションを書き込む
    set_annotation(doc_id, anno_name, annos)
close()


ボリビア映画協会
[{'begin': 8444, 'end': 8452}]
[]
[]
王立ブータン大学
Bhutan,通称:カンルン大学
[{'begin': 12587, 'end': 12595}, {'begin': 12623, 'end': 12639}]
プラハ大学
プラハ大学
カレル大学
[{'begin': 2148, 'end': 2153}, {'begin': 2242, 'end': 2247}, {'begin': 2248, 'end': 2253}]
[]
[]
[]
チリ大学
サンティアゴ・デ・チレ大学
チリ・カトリック大学
[{'begin': 22148, 'end': 22152}, {'begin': 22166, 'end': 22179}, {'begin': 22187, 'end': 22197}]
福島県立医科大学
ガーナ大学
ガーナ大学
クマシ大学
ケープ・コースト大学
ガーナサッカー協会
[{'begin': 4244, 'end': 4252}, {'begin': 4281, 'end': 4286}, {'begin': 7748, 'end': 7753}, {'begin': 7760, 'end': 7765}, {'begin': 7766, 'end': 7776}, {'begin': 9341, 'end': 9350}]
[]
エクアドル中央大学
グアヤキル大学
クエンカ大学
[{'begin': 11120, 'end': 11129}, {'begin': 11142, 'end': 11149}, {'begin': 11157, 'end': 11163}]
南太平洋大学
[{'begin': 3964, 'end': 3970}]
日本気象協会
日本気象協会
[{'begin': 15355, 'end': 15361}, {'begin': 15456, 'end': 15462}]
[]
ブカレスト大学
[{'begin': 5287, 'end': 5294}]
[]
キクユ青年協会
キクユ中央協会
キクユ中央協会
ナイロビ大学
[{'begin': 1317, 'end': 1324}, {'begin': 1394, 'end': 1401}, {'begin': 